# Security controls demo notebook
This notebook guides through a demo of different security controls.

## Infrastructure walk-through
Take a look to the following components and services created by the deployment:

- VPC setup
- Subnets - all private
- Network devices (NAT Gateway, Network Firewall) and the route tables
- Deployed security groups (SageMaker and VPC Endpoints security groups) and their ingress and egress rules
- S3 VPC endpoint setup with the endpoint policy
- S3 VPC interface endpoints for AWS public services
- S3 buckets named `<project_name>-<account_id>-<region>-models` and `<project_name>-<account_id>-<region>-data` with the bucket policy. You can try and see that there is no AWS console access to the solution buckets (data and models). When you try to list the bucket content in AWS console you will get `AccessDenied` exception
- AWS Network Firewall routing setup. You might want to read through [Deployment models for AWS Network Firewall](https://aws.amazon.com/blogs/networking-and-content-delivery/deployment-models-for-aws-network-firewall/) to familiarize yourself with different types of AWS Network Firewall deployment
- Firewall policy with a stateful rule group with an allow domain list. There is a single domain `.kaggle.com` on the list
- SageMaker IAM execution role
- KMS CMKs for EBS and S3 bucket encryption

## S3 access 
Create a file:

In [ ]:
data_bucket = '<project_name>-<account_id>-<region>-data'
model_bucket = '/<project_name>-<account_id>-<region>-models'

In [ ]:
!touch test-file.txt

Copy file to `data` S3 bucket. The operation must be successful:

In [ ]:
!aws s3 cp test-file.txt s3://{data_bucket}

Try to copy the file to any other bucket or list any other bucket. You'll get an `AccessDenied` exception: `An error occurred (AccessDenied) when calling the PutObject operation: Access Denied`:

In [ ]:
!aws s3 cp test-file.txt s3://<any other bucket in your account>

Try to list `-data` or `-models` S3 buckets. The operation must be successful.

In [ ]:
 !aws s3 ls s3://{data_bucket}

Try to list any other bucket in our account. You'll get an `AccessDenied` exception: `An error occurred (AccessDenied) when calling the ListObjectsV2 operation: Access Denied`:

In [ ]:
!aws s3 ls s3://<any other bucket in your account>

Try to list the `<project_name>-<account_id>-<region>-data` bucket from a command line outside of Studio. You'll get an `AccessDenied` error.

SageMaker Studio has access to the designated S3 buckets (`-models` and `-data`) and to these S3 buckets only. The access to S3 buckets is controlled by a combination of the S3 VPC endpoint policy and the S3 bucket policy.

❗ Note, you are not able to use [SageMaker JumpStart](https://docs.aws.amazon.com/sagemaker/latest/dg/studio-jumpstart.html) or any other SageMaker Studio functionality which requires access to other Amazon S3 buckets. To enable access to other S3 buckets you have to change the S3 VPC endpoint policy.

### Enable access to JumpStart S3 buckets
Now we are going to change the S3 VPC endpoint policy and to allow access to additional S3 resources.

First, try to open SageMaker Studio JumpStart:
![SageMaker Studio JumpStart](../design/sm-studio-jumpstart.png)

The access is denied because the S3 VPC endpoint policy doesn't allow access to any S3 buckets except for `models` and `data` as configured in the endpoint policy:
![S3 access denied](../design/jumpstart-no-access-s3.png)

Now add the following statement to the S3 VPC endpoint policy:
```json
{
  "Effect": "Allow",
  "Principal": "*",
  "Action": [
    "s3:GetObject"
  ],
  "Resource": "*",
  "Condition": {
    "StringEqualsIgnoreCase": {
      "s3:ExistingObjectTag/SageMaker": "true"
    }
  }
}
```

Command line:
```sh
cat <<EoF >s3-vpce-policy.json
{
    "Effect": "Allow",
    "Principal": "*",
    "Action": [
    "s3:GetObject"
    ],
    "Resource": "*",
    "Condition": {
    "StringEqualsIgnoreCase": {
        "s3:ExistingObjectTag/SageMaker": "true"
    }
    }
}
EoF
```

```sh
VPCE_ID=# VPC Id from the stack output

aws ec2 modify-vpc-endpoint \
    --vpc-endpoint-id $VPCE_ID \
    --policy-document file://s3-vpce-policy.json
```

Refresh the JumpPage start page - now you have access to all JumpStart resources

We have seen now, that you can control access to S3 buckets via combination of S3 bucket policy and S3 Endpoint policy.

## Control internet traffic

This operation succeeds because the domain `.kaggle.com` is allowed by the firewall policy. You can connect to this and only to this domain from any Studio notebook.

In [ ]:
!wget https://kaggle.com

This operation times out after 5 minutes because any internet traffic except to and from the `.kaggle.com` domain isn’t allowed and is dropped by Network Firewall.

In [ ]:
!git clone https://github.com/aws-samples/amazon-sagemaker-studio-vpc-networkfirewall.git

Add `.github.com` domain to the allowd domain list.

In [ ]:
!git clone https://github.com/aws-samples/amazon-sagemaker-studio-vpc-networkfirewall.git